In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
from seaborn import histplot, pointplot, lineplot
import scipy
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from matplotlib.transforms import Affine2D
import random
import copy
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.utils.datasets import get_dataset_path
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
import numpy as np

from traitlets.config import Config
from ctapipe.io import EventSource
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
import matplotlib.pyplot as plt
from ctapipe.utils import get_dataset_path
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
import ctapipe
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator

In [ ]:
# CLASSIC ANALYSIS
filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run201_muon.simtel.gz'
source = EventSource(filename, max_events=10000)
all_muon_parameters = ["optical_efficiency"]

events = []  
bad_events_obs21 = [7258,4400] 
#bad_events_obs1  = [7496]
bad_events_obs2  = [1096, 876, 3649, 6122]
norm_events = [2402]
event_iterator = iter(source)
for i,j in enumerate(event_iterator):
    if i > 2000:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        #print("small ", j.dl1.tel[1].parameters.morphology.n_small_islands)
        #print("medium ", j.dl1.tel[1].parameters.morphology.n_medium_islands)
        #print("large ", j.dl1.tel[1].parameters.morphology.n_large_islands)
        #print("ring width ", j.muon.tel[1].efficiency.width)
        plt.figure()
        camgeom = source.subarray.tel[1].camera.geometry
        #print("width ", j.muon.tel[1].efficiency.width)
        title="CT{}, event {} cog {} fit {} width {}".format(1,j.index.obs_id, round(j.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title)
        #disp.image = event.r0.tel[1].waveform[0].sum(axis=1)
        disp.image = j.simulation.tel[1].true_image #.sum(axis=1)
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #print("efficiency ", j.muon.tel[1].efficiency.optical_efficiency)
        #print(j.muon.tel[1].efficiency)
        print("cog ", j.dl1.tel[1].parameters.concentration.cog)
        #plt.show()
        event = j
        event.index.event_id
        #break
        #print(i)
        if j.muon.tel[1].efficiency.width.to_value() > 0.12:
            plt.savefig(f"/Users/vdk/MuonAnalysisImages/obs201TrueImages/TrueImage{i}")
        #break
        if i == 9970:
            event = j
            break
plt.close()

In [ ]:
# CLASSIC ANALYSIS
filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run201_muon.simtel.gz'
source = EventSource(filename, max_events=9000)
all_muon_parameters = ["optical_efficiency"]

events = []  
bad_events_obs21 = [7258,4400] 
#bad_events_obs1  = [7496]
bad_events_obs2  = [1096, 876, 3649, 6122]
norm_events = [2402]
event_iterator = iter(source)
for i,j in enumerate(event_iterator):
    if i == 200:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        #print("small ", j.dl1.tel[1].parameters.morphology.n_small_islands)
        #print("medium ", j.dl1.tel[1].parameters.morphology.n_medium_islands)
        #print("large ", j.dl1.tel[1].parameters.morphology.n_large_islands)
        #print("ring width ", j.muon.tel[1].efficiency.width)
        plt.figure()
        camgeom = source.subarray.tel[1].camera.geometry
        print("width ", j.muon.tel[1].efficiency.width)
        title="CT{}, run {} cog {} fit {} width {}".format(1,j.index.event_id, round(j.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title)
        #disp.image = event.r0.tel[1].waveform[0].sum(axis=1)
        disp.image = j.simulation.tel[1].true_image #.sum(axis=1)
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        print("efficiency ", j.muon.tel[1].efficiency.optical_efficiency)
        #print(j.muon.tel[1].efficiency)
        print("cog ", j.dl1.tel[1].parameters.concentration.cog)
        #plt.show()
        event = j
        event.index.event_id
        #break
        print(i)
        #if j.muon.tel[1].efficiency.width.to_value() > 0.1:
        plt.savefig(f"/Users/vdk/TrueImage{i}")
        #break
        if i == 200:
            event = j
            break
plt.close()

In [ ]:
# CHECK FOR COG DEPENDENCE

#cl = JSONFileConfigLoader(filename = "throughput_configuration_json.json", path =  '~/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/')
#cl._find_file()
#config = cl._read_file_as_dict()
#config
#config_file = "~/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/throughput_configuration.yaml"
#path = pathlib.Path("~/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/throughput_configuration.yaml")
#image_processor = ImageProcessor(source.subarray, config = Config({"~/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/throughput_configuration.yaml"}))
#conf = Config({"~/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/throughput_configuration.yaml"})
#image_processor = ImageProcessor(source.subarray, config = conf)


filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run201_muon.simtel.gz'
source = EventSource(filename, max_events=9000)

events = []  
bad_events_obs21 = [7258,4400] 
#bad_events_obs1  = [7496]
bad_events_obs2  = [1096, 876, 3649, 6122]
norm_events = [2402]
event_iterator = iter(source)
checker = 0
for i,j in enumerate(event_iterator):
    if 700 < i < 2000:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        if j.muon.tel[1].efficiency.width.tu_value > 10: 
        cog = j.dl1.tel[1].parameters.concentration.cog
        opt_eff = j.muon.tel[1].efficiency.optical_efficiency
        if not (np.isnan(opt_eff)):
            checker += 1
        if cog > 0 and not np.isnan(opt_eff):
            plt.figure()
            camgeom = source.subarray.tel[1].camera.geometry
            title="CT{}, run {} cog {} fit {} width {}".format(1,j.index.obs_id, round(j.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
            disp = CameraDisplay(camgeom,title=title)
            disp.image = j.simulation.tel[1].true_image #.sum(axis=1)
            disp.cmap = plt.cm.RdBu_r
            disp.add_colorbar()
            disp.set_limits_percent(95)
            plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}")
        if i == 4000:
            break

plt.close()

In [ ]:
# TO FIND BIG WIDTH
filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run101_muon.simtel.gz'
source = EventSource(filename, max_events=9000)

events = []  
bad_events_obs21 = [7258,4400] 
bad_events_obs2  = [1096, 876, 3649, 6122]
norm_events = [2402]
event_iterator = iter(source)
checker = 0
bad_eff_obs2= [254,  346,  382,  658,  876,  971,  999, 1096, 1537]
for i,j in enumerate(event_iterator):
    if i==4400:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        print("width = ", j.muon.tel[1].efficiency.width.to_value())
        #if j.muon.tel[1].efficiency.width.to_value() > 10: 
        cog = j.dl1.tel[1].parameters.concentration.cog
        opt_eff = j.muon.tel[1].efficiency.optical_efficiency
        #if not np.isnan(opt_eff):
        plt.figure()
        camgeom = source.subarray.tel[1].camera.geometry
        title="CT{}, run {} cog {} fit {} width {}".format(1,j.index.obs_id, round(j.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title)
        disp.image = j.simulation.tel[1].true_image #.sum(axis=1)
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        print(j.index.event_id)
        #plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}_COG=0_fitWrong")
        plt.show()
        event = j
        break
        #if i == 4000:
        #    break

plt.close()

In [ ]:
for i in range(len(event.dl1.tel[1].image)):
    print("image = ", event.dl1.tel[1].image[i], " mask = ", event.dl1.tel[1].image_mask[i])

In [ ]:
mask = np.array([True,True,False])
arr = np.array([1,2,3])

arr[mask]

In [ ]:
# TO FIND check efficiency distribution
filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run201_muon.simtel.gz'
source = EventSource(filename, max_events=9000)

events = []  
bad_events_obs21 = [7258,4400] 
bad_events_obs2  = [1096, 876, 3649, 6122]
norm_events = [2402]
event_iterator = iter(source)
checker = 0
optical_efficiency = []
for i,j in enumerate(event_iterator):
    if i < 2000:
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        cog = j.dl1.tel[1].parameters.concentration.cog
        #if cog == 0:
        optical_efficiency.append(j.muon.tel[1].efficiency.optical_efficiency)
        #if not np.isnan(opt_eff):
        checker += 1
        if i == 2000:
            break

plt.close()

In [ ]:
optical_efficiency = np.array(optical_efficiency)
mean = np.mean(optical_efficiency)
x_values = [i for i in range(len(optical_efficiency))]
plt.scatter(x_values, optical_efficiency)
np.where(optical_efficiency > 0.4)

In [ ]:
print("small ", event.dl1.tel[1].parameters.morphology.n_small_islands)
print("medium ", event.dl1.tel[1].parameters.morphology.n_medium_islands)
print("large ", event.dl1.tel[1].parameters.morphology.n_large_islands)


In [ ]:
event.dl1.tel[1].parameters.concentration.cog

In [ ]:
disp = CameraDisplay(camgeom,title=title)
#disp.image = event.r0.tel[1].waveform[0].sum(axis=1)
disp.image = event.r0.tel[1].waveform[0].sum(axis=1) #.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
#plt.savefig("/Users/vdk/WaveFormEvent1096")

In [ ]:
filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run201_muon.simtel.gz'
source = EventSource(filename, max_events=9000)
all_muon_parameters = ["optical_efficiency"]

events = []  
bad_events_obs21 = [7258,4400] 
#bad_events_obs1  = [7496]
bad_events_obs2  = [1096, 876, 3649, 6122]
event_iterator = iter(source)
for i,j in enumerate(event_iterator):
    if i == 1096:
        event = j
        
disp = CameraDisplay(camgeom,title=title)
#disp.image = event.r0.tel[1].waveform[0].sum(axis=1)
disp.image = event.r0.tel[1].waveform[0].sum(axis=1) #.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

In [ ]:
filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run201_muon.simtel.gz'
source = EventSource(filename, max_events=9000)
all_muon_parameters = ["optical_efficiency"]

events = []  
bad_events_obs21 = [7258,4400] 
#bad_events_obs1  = [7496]
bad_events_obs2  = [1096, 876, 3649, 6122]
event_iterator = iter(source)
for i,j in enumerate(event_iterator):
    if i == 2402:
        event = j
        
disp = CameraDisplay(camgeom,title=title)
#disp.image = event.r0.tel[1].waveform[0].sum(axis=1)
disp.image = event.r0.tel[1].waveform[0].sum(axis=1) #.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)

In [ ]:
muon_ring = ascii.read("/Users/vdk/UncertaintyEstimations/bigdataset_obs101Containers/MuonRing_obs101_minPixel70.txt")
center_distance = np.array(muon_ring[0::5]['col3'])
center_fov_lat = np.array(muon_ring[1::5]['col3'])
center_fov_lon = np.array(muon_ring[2::5]['col3'])
center_phi = np.array(muon_ring[3::5]['col3'])
radius = np.array(muon_ring[4::5]['col3'])
len(center_fov_lon)
x_values = [i for i in range(0,8452)]
plt.figure(figsize=(12,8))
plt.scatter(x_values, radius+center_distance)
plt.xlabel("Events")
plt.ylabel("Degrees")
plt.title("Distance of farther point of the ring from camera center")

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10,5))
histplot(radius, ax = axes[0])
histplot(center_distance, ax = axes[1])
axes[0].set_xlabel('degrees')
axes[1].set_xlabel('degrees')
axes[0].set_title("ring radius")
axes[1].set_title("ring center distance")




In [ ]:
filename1 = '/Users/vdk/UncertaintyEstimations/bigdataset/Completeness_obs101_minPixel70.txt'
completeness = []
with open(filename1) as file:
    for line in file:
        completeness.append(line)

completeness = [float(i) for i in completeness]

filename2 = '/Users/vdk/UncertaintyEstimations/bigdataset/Containment_obs101_minPixel70.txt'
containment = []
with open(filename2) as file:
    for line in file:
        containment.append(line)

containment = [float(i) for i in containment]
plt.scatter(x_values, completeness)

In [ ]:
plt.scatter(x_values, containment)

In [ ]:
filename3 = '/Users/vdk/UncertaintyEstimations/bigdataset/Energy_obs101_minPixel70.txt'
energy = []
with open(filename3) as file:
    for line in file:
        energy.append(line)

energy = [float(i) for i in energy]

In [ ]:
plt.scatter(x_values, energy)

In [ ]:
filename = '/Users/vdk/UncertaintyEstimations/bigdataset/Width_obs101_minPixel70.txt'
ring_width = []

with open(filename) as file:
    for line in file:
        ring_width.append(float(line))
        
plt.rcParams.update({'font.size': 12, 'figure.figsize': (12,12)})
plt.figure(figsize=(12,8))
plt.scatter(x_values, ring_width)
plt.ylim()
plt.grid()
plt.yticks([0,0.5, 1,2,4,6,8,10,12])
plt.xlabel("Events")
plt.ylabel("Degrees")
plt.title("Ring width on the mirror")

In [ ]:
for i,j in enumerate(ring_width):
    if j > 1:
        print(i)
        print(j)

In [ ]:
filename1 = '/Users/vdk/UncertaintyEstimations/bigdataset/Completeness_obs101_minPixel70.txt'
completeness = []
with open(filename1) as file:
    for line in file:
        completeness.append(line)

completeness = [float(i) for i in completeness]

good_completeness = []
for i,j in enumerate(completeness):
    if j >= 0.9:
        good_completeness.append(i)

filename2 = '/Users/vdk/UncertaintyEstimations/bigdataset/Containment_obs101_minPixel70.txt'
containment = []
with open(filename2) as file:
    for line in file:
        containment.append(line)

containment = [float(i) for i in containment]

good_containment = []
for i,j in enumerate(containment):
    if j >= 1:
        good_containment.append(i)

        
filename = '/Users/vdk/UncertaintyEstimations/bigdataset_9956Events/Efficiency_obs101_minPixel70.txt'
optical_efficiency = []
N = 11
N_LEN = 845
count = 0
drop_count = 6360


with open(filename) as file:
    for line in file:
        if count != drop_count:
            if count in good_completeness:
                if count in good_containment:
                    optical_efficiency.append(float(line))
        count += 1

print("len = ", len(optical_efficiency))

mean_arr = []
std_arr = []
rel_un_arr = []
for i in range(1,N):
    mean_arr.append(np.mean(optical_efficiency[0:i*N_LEN]))
    std_arr.append(np.std(optical_efficiency[0:i*N_LEN]))
    rel_un_arr.append(np.std(optical_efficiency[0:i*N_LEN])/np.mean(optical_efficiency[0:i*N_LEN]))
    #print("len = ", len(optical_efficiency[0:i*N_LEN]))

#print(mean_arr)
#print(std_arr)
#print(rel_un_arr)
relative_uncertainty = np.array(rel_un_arr)*100
plt.rcParams.update({'font.size': 22, 'figure.figsize': (12,12)})
events_number = [i*N_LEN for i in range(1,N)]

fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
fig.subplots_adjust(hspace=0.0)
ax = axes[0]
ax.errorbar(events_number, mean_arr, yerr=std_arr, fmt='o', ms = 10)#, c = 'blue')#, s = 100)#, c= 'black')
#ax.set_title('Optical efficiency together with its relative uncertainty')
ax.set_title('Optical efficiency for obs101 with min pixel>70')
ax.set_ylabel('Optical efficiency')
ax.set_ylim(0.01,0.4)
bx = axes[1]
bx.scatter(events_number, relative_uncertainty, marker = '+', s = 350, linewidths = 3.5)#, sharex = ax)#, c = "black")
#bx.annotate('added events from obs901', xy=(5850, 15.7), xytext=(4500, 17), color= 'red', size = 15,
            #arrowprops=dict(facecolor='orange', shrink=0.005))
#bx.annotate('added events from obs901', xy=(5550, 15), xytext=(4500, 17), color= 'red', size = 15,
            #arrowprops=dict(facecolor='orange', shrink=0.005))

#bx.annotate('events from obs201', xy=(424, 18), xytext=(600, 12), color= 'red', size = 15,
            #arrowprops=dict(facecolor='orange', shrink=0.005))
#bx.annotate('events from obs201', xy=(848, 18), xytext=(600, 12), color= 'red', size = 15,
            #arrowprops=dict(facecolor='orange', shrink=0.005))
    
bx.set_ylabel("Rel. Unc.")
bx.set_xlabel("Cumulative number of events")
bx.set_ylim(10,20)
#bx.set_title('Relative uncertainty (in %)')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
#bx.set_yticks([11,12,13,14,15,16,17,18,19,20])
bx.tick_params(axis="y", labelsize=13)
plt.savefig("/Users/vdk/Uncertainty_test1.png")

print("mean = ", np.mean(optical_efficiency))
print("std = ", np.std(optical_efficiency))

In [ ]:
filename = '/Users/vdk/UncertaintyEstimations/bigdataset_9956Events/Efficiency_obs101_minPixel70.txt'
optical_efficiency = []
N = 11
N_LEN = 845
count = 0
drop_count = 6360


with open(filename) as file:
    for line in file:
        if count != drop_count:
            #if count in good_completeness:
            optical_efficiency.append(float(line))
        count += 1

print("len = ", len(optical_efficiency))

mean_arr = []
std_arr = []
rel_un_arr = []
for i in range(1,N):
    mean_arr.append(np.mean(optical_efficiency[0:i*N_LEN]))
    std_arr.append(np.std(optical_efficiency[0:i*N_LEN]))
    rel_un_arr.append(np.std(optical_efficiency[0:i*N_LEN])/np.mean(optical_efficiency[0:i*N_LEN]))
    #print("len = ", len(optical_efficiency[0:i*N_LEN]))

#print(mean_arr)
#print(std_arr)
#print(rel_un_arr)
relative_uncertainty = np.array(rel_un_arr)*100
plt.rcParams.update({'font.size': 22, 'figure.figsize': (12,12)})
events_number = [i*N_LEN for i in range(1,N)]

fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
fig.subplots_adjust(hspace=0.0)
ax = axes[0]
ax.errorbar(events_number, mean_arr, yerr=std_arr, fmt='o', ms = 10)#, c = 'blue')#, s = 100)#, c= 'black')
#ax.set_title('Optical efficiency together with its relative uncertainty')
ax.set_title('Optical efficiency for obs101 with min pixel>70')
ax.set_ylabel('Optical efficiency')
ax.set_ylim(0.01,0.4)
bx = axes[1]
bx.scatter(events_number, relative_uncertainty, marker = '+', s = 350, linewidths = 3.5)#, sharex = ax)#, c = "black")
#bx.annotate('added events from obs901', xy=(5850, 15.7), xytext=(4500, 17), color= 'red', size = 15,
            #arrowprops=dict(facecolor='orange', shrink=0.005))
#bx.annotate('added events from obs901', xy=(5550, 15), xytext=(4500, 17), color= 'red', size = 15,
            #arrowprops=dict(facecolor='orange', shrink=0.005))

#bx.annotate('events from obs201', xy=(424, 18), xytext=(600, 12), color= 'red', size = 15,
            #arrowprops=dict(facecolor='orange', shrink=0.005))
#bx.annotate('events from obs201', xy=(848, 18), xytext=(600, 12), color= 'red', size = 15,
            #arrowprops=dict(facecolor='orange', shrink=0.005))
    
bx.set_ylabel("Rel. Unc.")
bx.set_xlabel("Cumulative number of events")
bx.set_ylim(10,20)
#bx.set_title('Relative uncertainty (in %)')
ax.grid(alpha = 0.5)
bx.grid(alpha = 0.5)
#bx.set_yticks([11,12,13,14,15,16,17,18,19,20])
bx.tick_params(axis="y", labelsize=13)
plt.savefig("/Users/vdk/Uncertainty_test1.png")


print("mean = ", np.mean(optical_efficiency))
print("std = ", np.std(optical_efficiency))

In [ ]:
filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run201_muon.simtel.gz'
source = EventSource(filename, max_events=9000)
all_muon_parameters = ["optical_efficiency"]

#for event in source:
    #print(event.simulation.shower.energy)
    #print(event.keys())
    #print(event.muon.tel[1].ring.radius)
    #print(event.simulation.shower.az)
    #print(event.simulation.shower.az.degree)
    
bad_events_obs21 = [7258,4400] 
#bad_events_obs1  = [7496]
bad_events_obs2  = [1096, 876, 3649, 6122]
event_iterator = iter(source)
for i,j in enumerate(event_iterator):
    if i == 3649:
        event = j
        break

In [ ]:
plt.figure()
camgeom = source.subarray.tel[1].camera.geometry
title="CT{}, run {} event {}".format(1,event.index.obs_id,event.index.event_id)
disp = CameraDisplay(camgeom,title=title)
disp.image = event.r0.tel[1].waveform[0].sum(axis=1)
#disp.image = event.simulation.tel[1].true_image #.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
#plt.savefig("/Users/vdk/RawImageEvent6122")


In [ ]:
33/6600

In [ ]:
type(event.dl1.tel[1].image)

In [ ]:
good+380 (95) 489 26 48_muon_2402_1 = [5482,3504,3428.256,3421.99,3418.443,3418.424, 3418.418, 3418.418]
N_calls = [110,128,144,159,172, 186, 333, 370]
Edm = [1429.05,26.65,1.7,2.64,0.21,0.012,0.0012,0.0001]


bad_muon_1096_1 = [15811.4, 14632,13197, 12974, 12933,12911,12911]
N_calls_1096 = [15,47,60,73,123,160,207]
Edm_1096 = [2007,1494,85.2,12,11,0.004, 7e-6]


plt.rcParams.update({'font.size': 13, 'figure.figsize': (12,12)})

fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
fig.subplots_adjust(hspace=0.0)
ax = axes[0]
ax.grid()
ax.set_title("Fit convergence for good muon ring")
ax.scatter(N_calls,good_muon_2402_1, s = 150)
ax.set_ylabel('Cost function (log-likelihood')
bx = axes[1]
bx.grid()
bx.scatter(N_calls,Edm, s = 150)
bx.annotate('Required Edm reached (1e-4)', xy=(367, 50), xytext=(250, 500), color= 'red', size = 15, 
            arrowprops=dict(facecolor='orange', shrink=0.005))
bx.set_ylabel('Estimate distance to minimum')
bx.set_xlabel('Number of calls')

plt.savefig('/Users/vdk/test.png')

In [ ]:
bad_muon_1096_1 = [15811.4, 14632,13197, 12974, 12933,12911,12911]
N_calls_1096 = [15,47,60,73,123,160,207]
Edm_1096 = [2007,1494,85.2,12,11,0.004, 7e-6]


plt.rcParams.update({'font.size': 13, 'figure.figsize': (12,12)})

fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
fig.subplots_adjust(hspace=0.0)
ax = axes[0]
ax.grid()
ax.set_title("Fit convergence for bad muon ring (converge to boundary)")
ax.scatter(N_calls_1096,bad_muon_1096_1, s = 150)
ax.set_ylabel('Cost function (log-likelihood')
bx = axes[1]
bx.grid()
bx.scatter(N_calls_1096,Edm_1096, s = 150)
bx.annotate('Required Edm reached (!Edm = 1e-7)', xy=(207, 50), xytext=(110, 530), color= 'red', size = 15, 
            arrowprops=dict(facecolor='orange', shrink=0.005))
bx.set_ylabel('Estimate distance to minimum')
bx.set_xlabel('Number of calls')

plt.savefig('/Users/vdk/test2.png')

In [ ]:
print(event.muon.tel[1].efficiency)

In [ ]:
print(event.muon.tel[1].efficiency)

In [ ]:
filename = '/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/calibpipe/tests/data/throughput/output/simtel_run201_muon.simtel.gz'
source = EventSource(filename, max_events=9000)
all_muon_parameters = ["optical_efficiency"]

#for event in source:
    #print(event.simulation.shower.energy)
    #print(event.keys())
    #print(event.muon.tel[1].ring.radius)
    #print(event.simulation.shower.az)
    #print(event.simulation.shower.az.degree)
    
bad_events_obs21 = [7258,4400] 
#bad_events_obs1  = [7496]
bad_events_obs2  = [1096, 876, 3649, 6122]
event_iterator = i+380 (95) 489 26 48ter(source)
for i,j in enumerate(event_iterator):
    print("alt = ", j.simulation.shower.alt.to_value('deg'))
    #print("az = ", j.simulation.shower.az.to_value('deg'))
    if i == 3000:
        event = j
        break

In [ ]:
#print(event.simulation.shower.alt)
#print(event.simulation.shower.az)
event.dl1.tel[1].image

In [ ]:
arr = np.array([1,2,3,4,5])
bad_arr = np.array([0,1,2,3])

arr[~bad_arr]

In [ ]:
arr = [1,2]
arr2 = [3,4,0]
np.size(arr and arr2)

In [ ]:
0.217+2*0.015

In [ ]:
obs = 2
minPix = 100
print(obs)
filename1 = f'/Users/vdk/UncertaintyEstimations/bigdataset/Width_obs{obs}01_minPixel{minPix}.txt'
filename2 = f'/Users/vdk/UncertaintyEstimations/bigdataset/Radius_obs{obs}01_minPixel{minPix}.txt'
ring_width = []
radius = []

count = 0


with open(filename1) as file:
    for line in file:
            ring_width.append(float(line))
            count += 1

with open(filename2) as file:
    for line in file:
            radius.append(float(line)) 

            
print(len(ring_width))
print(len(radius))

relation1 = [np.array(radius)/np.array(ring_width)]
relation2 = [np.array(ring_width)/np.array(radius)]

x_values = [i for i in range(count)]

plt.rcParams.update({'font.size': 12, 'figure.figsize': (6,6)})
plt.figure(figsize=(12,8))
plt.scatter(x_values, ring_width)
plt.ylim()
plt.grid()

#plt.yticks([0,0.5, 1,2,4,6,8,10,12])

plt.xlabel("Events")
plt.ylabel("Degrees")
plt.title("Radius")
#print("MEAN = ",np.mean(ring_width))
#print("STD = ",np.std(ring_width))

plt.savefig(f"/Users/vdk/ЦшверRel_obs{obs}01_minPix>{minPix}.png")

In [ ]:
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10,6)})
histplot(relation2, bins = 100)

In [ ]:
np.mean(ring_width) - np.std(ring_width)*2

In [ ]:
np.mean(relation2)-np.std(relation2)*2

In [ ]:
filename1 = '/Users/vdk/energies_500.txt'
fi1 = open(filename1,'r')
lines = fi1.readlines() 
splitted = lines[0].split(',')
energies_proton = np.array(list(filter(None, splitted))).astype('float')

In [ ]:
filename2 = '/Users/vdk/energies_for_filerange1-499.txt'
fi2 = open(filename2, 'r')
lines = fi2.readlines()
energies_muons = np.array(lines).astype('float')


In [ ]:
histplot(energies_proton, log= True, bins = 45)


In [ ]:
histplot(energies_muons, log= True, bins = 7)

In [ ]:
w_protons = [i**(-0.7) for i in energies_proton]
w_protons.sort(reverse=True)
energies_proton = list(energies_proton)
energies_proton.sort(reverse=True)
sum(w_protons)
len(energies_proton)/sum(w_protons)
#type(energies_proton)

In [ ]:
#for i,j in enumerate(sorted(energies_proton)):
#    print("old energy = ", j, "new energy = ", sorted(w_protons)[i])

for i,j in enumerate(energies_proton):
    print("old energy = ", j, "new energy = ", w_protons[i])

In [ ]:
w_muons = [energy**(-0.7) for energy in energies_muons]
len(energies_muons)/sum(w_muons)

In [ ]:
len(energies_muons)/sum(w_muons) * len(energies_muons)

In [ ]:
(len(energies_muons)/sum(w_muons) * len(energies_muons))/len(energies_proton)/sum(w_protons) * len(energies_proton)

In [ ]:
print(1/(20**(-0.7)))



In [ ]:

for i,j in enumerate(sorted(energies_muons)):
    print("old energy = ", j, "new energy = ", sorted(w_muons)[i])

In [ ]:
particle = utils.guess_type(input_filename)

K = numevents*(1+index)/(emax**(1+index)-emin**(1+index))                    

A = np.pi*core_max**2
Omega = 2*np.pi*(1-np.cos(cone))
if cone==0:
    Omega=1
MeVtoGeV = 1e-3                                                     
if particle=="gamma":                                                
    K_w = 5.7e-16*MeVtoGeV                                           
    index_w = -2.48                                                  
    E0 = 0.3e6*MeVtoGeV                                              
                                                                         
if particle=="proton":                                               
    K_w = 9.6e-2                                                     
    index_w = -2.7                                                   
    E0 = 1                                                           
                                                                         
Simu_E0 = K*E0**index                                                
N_ = Simu_E0*(emax**(index_w+1)-emin**(index_w+1))/(E0**index_w)/(index_w+1)                                                                 
R = K_w*A*Omega*(emax**(index_w+1)-emin**(index_w+1))/(E0**index_w)/(index_w+1)

energy = event.mc.energy.value

w = ((energy)**(index_w-index))*R/N_

print('Spectral weight: ',w)

In [ ]:
import  numpy as np
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
import numpy as np
from ctapipe.io import EventSource
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.calib import CameraCalibrator


int_cut = 0.3
min_pixel = 50

muon_processor_config = Config({"MuonProcessor":
                                {"RingQuery":
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

all_events = 0
stereo_events = 0
muon_ring_number = 0
event_number = []

start = 3980
stop = 3981

for k in range(start,stop):
    filename = f'/Users/vdk/LST/LST_work/corsika_4LSTprotons/simtel_corsika_run{k}.simtel.gz'
    #print(filename)
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,j in enumerate(event_iterator):
        #print("process events", i)
        all_events += 1
        if len(j.trigger.tels_with_trigger) > 1:
            stereo_events += 1
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            #calib(j)
            #image_processor(j)
            #muon_processor(j)
            event_number.append(str(k)+str(i))
            for m in j.muon.tel.keys():
                if not np.isnan(j.muon.tel[m].efficiency.width):
                    muon_ring_number += 1
                    break
    #np.savetxt(f'/fefs/aswg/workspace/vadym.voitsekhovskyi/muon_rings_for_runs{start}-{stop}_4LST_allevents.txt', [all_events])
    #np.savetxt(f'/fefs/aswg/workspace/vadym.voitsekhovskyi/muon_rings_for_runs{start}-{stop}_4LST_stereo.txt', [stereo_events])
    #np.savetxt(f'/fefs/aswg/workspace/vadym.voitsekhovskyi/muon_rings_for_runs{start}-{stop}_4LST_muons.txt', [muon_ring_number])
    #np.savetxt(f'/fefs/aswg/workspace/vadym.voitsekhovskyi/muon_rings_for_runs{start}-{stop}_4LST_runs.txt', [k])
    print(event_number)
    np.savetxt(f'/Users/vdk/muon_rings_for_runs{start}-{stop}_4LST_number.txt', event_number, fmt='%s')

In [ ]:
source.subarray.peek()

In [ ]:
for i in range(0,171):
    print(eval(str(4-len(str(i))))*'0' + str(i))

In [ ]:
import math
def estimate_pi():
    pika = 0
    ka = 0
    summa = 0 
    while True:
        pika = 2 * math.sqrt(2) / 9801 * (math.factorial(4 * ka) * (1103 + 26390 * ka) / (math.factorial(ka) ** 4 * 396 ** (4 * ka)))
        if pika >= 1e-15:
            ka += 1
            summa += pika
        else:
            return summa
estimate_pi()

In [ ]:
def is_palindrome(text):
    return text == text[::-1]
is_palindrome(input("Enter your phrase (or word) without spaces to verify if it's a polyndrome"))

In [ ]:
 #Вправа №2
a="J'aime l'algebre linaire"
b=[]
shift=1
for i in a:
    b.append(chr((ord(i)+shift)%128))
print(''.join(b))
c=[]
for i in b:
    c.append(chr((ord(i)-shift)%128))
print(''.join(c))